### In this lab, we will continue to work on the openAI agents SDK. We will create three new agents which will take on the role of sales people of
### different personalities. the agents will be tasked to generate a cold sales email and then we will have a 4th agent which will take on the role of
### a customer and choose the best email that is likely to get a response.

In [1]:
from dotenv import load_dotenv
import asyncio
import os
from agents import Runner, Agent, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
from sendgrid.helpers.mail import Email, To, Content, Mail

In [2]:
load_dotenv(override=True)

True

In [3]:
instructions_1 = f'You are a sales agent working with CyberAI, a company which provides cyber security solutions for network security, endpoint security, \
    threat intelligence & detection and Identity and Access Management (IAM), powered by AI. You write professional, serious cold emails.'

instructions_2 = f'You are a humorous, engaging sales agent working with CyberAI, a company which provides cyber security solutions for network security, \
    endpoint security, threat intelligence & detection, and Identity and Access Management (IAM), powered by AI. You write witty, engaging cold emails \
        that are likely to get a response.'

instructions_3 = f'You are a sales agent working with CyberAI, a company which provides cyber security solutions for network security, endpoint security, \
    threat intelligence & detection, and Identity and Access Management (IAM), powered by AI. You write concise, to the point cold emails.'



In [4]:
# lets create an agent and try to generate a cold email
sales_agent_1 = Agent(name='Professional Sales Agent', instructions=instructions_1, model='gpt-4o-mini')
sales_agent_2 = Agent(name='Humorous Sales Agent', instructions=instructions_2, model='gpt-4o-mini')
sales_agent_3 = Agent(name='Concise Sales Agent', instructions=instructions_3, model='gpt-4o-mini')


In [ ]:
# we are going to use a streaming api here. What does that mean? We are going to use Runner.run_streamed() instead of Runner.run()
# This returns a RunResultStreaming object. this object has a coroutine "stream_events()" which yields events as they are generated by the agent.
# we can iterate over these stream events which are of type StreamEvent. Each StreamEvent has a type and data attribute. You can check the openAI agents DSK
# for more details on the types of events.
result = Runner.run_streamed(sales_agent_1, input = "Write a cold sales email.")
async for event in result.stream_events():
    if event.type == 'raw_response_event' and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end='', flush=True)


In [ ]:
# now, we will run these three agents in parallel and collect and print the results. This is very trivial.
with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent_1, input = "Write a cold sales email."),
        Runner.run(sales_agent_2, input = "Write a cold sales email."),
        Runner.run(sales_agent_3, input = "Write a cold sales email.")
    )
outputs = [result.final_output for result in results]

for output in outputs:
    print(f'{output}\n\n')

# you can go to platform.openai.com/traces to view the traces of the runs.

In [ ]:
# Now we will go one step further and create a new agent. this new agent will act as a customer and choose the best email from the three.
# for which he is likely to respond.
customer_instructions = f"You are a customer who is looking for a cyber security solution. You are given three cold emails. You need to choose the best email \
which is most likely to get a response. You will be given the emails and you need to choose the best one. Dont give an explanation for your choice. Just give me the email."

customer_agent = Agent(name='Customer', instructions=customer_instructions, model='gpt-4o-mini')

# now, we will run these three agents in parallel and collect and print the results. This is very trivial.
with trace("Sales picker"):
    results = await asyncio.gather(
        Runner.run(sales_agent_1, input = "Write a cold sales email."),
        Runner.run(sales_agent_2, input = "Write a cold sales email."),
        Runner.run(sales_agent_3, input = "Write a cold sales email.")
    )
    emails = ''
    for output in outputs:
        emails += f'{output}\n\n'

    best_email = await Runner.run(customer_agent, input = emails)
    print(f'The best email is: {best_email.final_output}')
    # you can go to platform.openai.com/traces to view the traces of the runs.

In [5]:
# We will know go one step further and use tools to generate and send emails.
# As you have seen in the previous lab, we created a lot of boilerplate json code to define the metadata for our tools,
# like the tool name, description, and argument names and types. All that is handled automatically by the agents SDK using the function_tool decorator.

@function_tool
def send_email(body: str):
    """Send a cold sales email with the given body to all sales prospects"""
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_API_KEY'))
    from_email = Email("rajat.girotra@gmail.com")
    to_email = To("rajatgirotra@yahoo.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get() # get returns a json object representing the email object which can be sent to the sendgrid api
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)
    return {'status': 'success'}



In [37]:
def try_send_email(body: str):
    """Send a cold sales email with the given body to all sales prospects"""
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_API_KEY'))
    from_email = Email("rajat.girotra@gmail.com")
    to_email = To("rajatgirotra@yahoo.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get() # get returns a json object representing the email object which can be sent to the sendgrid api
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)
    return {'status': 'success'}

try_send_email("Hello, how are you?")


202


{'status': 'success'}

In [6]:
send_email

FunctionTool(name='send_email', description='Send a cold sales email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x72947c5c25c0>, strict_json_schema=True)

In [7]:
# agents SDK can also convert an agent into a tool using the as_tool() method. Lets do that for all the sales agents.
description = 'Write a cold sales email to a customer'
tool1 = sales_agent_1.as_tool(tool_name='sales_agent1', tool_description=description)
tool2 = sales_agent_2.as_tool(tool_name='sales_agent2', tool_description=description)
tool3 = sales_agent_3.as_tool(tool_name='sales_agent3', tool_description=description)

tools = [tool1, tool2, tool3, send_email]

In [8]:
# Now create a new agent which uses all these tools.
# sales_manager_instructions = f'You are a sales manager working with CyberAI. you are responsible for sending cold emails to customers.\
#       You must use the tools provided to you to generate emails. You must not generate the email body yourself. You should use each of the 3 tools atleast once.\
#       Of the 3 emails generated, you must choose the best one which is likely to get a response. Dont give an explanation for your choice. Just give the email. \
#       Post that, use the send_email tool to send the email to the customer.'

sales_manager_instructions = "You are a sales manager working for CyberAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. You should ask each of the sales_agent tools to write a cold sales email. \
You try all 3 sales_agent tools once before choosing the best one. The input for each of the sales_agent tools should be 'Write a cold sales email'. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."

sales_manager_agent = Agent(name='Sales Manager', instructions=sales_manager_instructions, tools=tools, model='gpt-4o-mini')

with trace("Sales manager"):
    result = await Runner.run(sales_manager_agent, input = "Send a cold sales email addressed to 'Dear CEO'")

print(result.final_output)

202
The cold sales email has been successfully sent to 'Dear CEO'. If you need any more assistance or follow-ups, feel free to ask!


In [19]:
## Next, we will discuss about Handoffs. Handoffs are a way to delegate tasks to other agents. Its like a fire and forget mechanism.
## The agent which is delegating the task forgets about it and the other agent takes over.
## this is slightly different from using an agent as a tool. In a tool, the agent asks the other agent to do something and then waits for the other agent to finish, before it can continue.

### So what are we going to do here?
### We will create two new agents and use them as tools. The first agent just generates an email subject from the email body.
### the second agent converts a text email to an html email.
### we will also define a function tool "send_html_email", which is very similar to the send_email tool we defined earlier.

@function_tool
def send_html_email(subject: str, body: str):
    """Send an html email with the given subject and body to the user"""
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_API_KEY'))
    from_email = Email("rajat.girotra@gmail.com")
    to_email = To("rajatgirotra@yahoo.com")
    content = Content("text/html", body)
    mail = Mail(from_email, to_email, subject, content).get() # get() converts the email object to a json object
    sg.client.mail.send.post(request_body=mail)
    return {'status': 'success'}

In [10]:
send_html_email("Hello", "Hello, how are you?")

{'status': 'success'}

In [38]:
subject_writer_instructions = "You are a subject writer. You are given text of an email. You need to write an appropriate subject for the email. \
    The subject should be a single line sentence which captures the main idea of the email. The subject should be no more than 30 characters."

subject_writer_agent = Agent(name='Subject Writer', instructions=subject_writer_instructions, model='gpt-4o-mini')
subject_writer_tool = subject_writer_agent.as_tool(tool_name='subject_writer', tool_description="Subject writer tool")

html_writer_instructions = "You are an html writer. You are given a text email which can have some markdown in it and you need to convert it to an HTML email body \
      with simple, clear, compelling layout and design."
html_writer_agent = Agent(name='HTML Writer', instructions=html_writer_instructions, model='gpt-4o-mini')
html_writer_tool = html_writer_agent.as_tool(tool_name='html_writer', tool_description="HTML writer tool")

In [42]:
tools_for_email_writer = [subject_writer_tool, html_writer_tool, send_html_email]

In [43]:
tools_for_email_writer

[FunctionTool(name='subject_writer', description='Subject writer tool', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7294697e8a40>, strict_json_schema=True),
 FunctionTool(name='html_writer', description='HTML writer tool', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7294697e8540>, strict_json_schema=True),
 FunctionTool(name='send_html_email', description='Send an html email with the given subject and body to the user', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'body

In [44]:
email_writer_instructions = "You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email. The input to the subject_writer tool should be the text of the email. After using the subject_writer tool, \
you then use the html_writer tool to convert the body of the emailto HTML. \
Finally, you use the send_html_email tool to send the email with the subject output by the subject_writer tool and the HTML body output by the html_writer tool."

In [45]:
# Note this important step: We specify the handoff where.

email_writer_agent = Agent(name='Email Writer',
                        instructions=email_writer_instructions,
                        tools=tools_for_email_writer,
                        model='gpt-4o-mini',
                        handoff_description="Convert an email to HTML and send it.")
# this agent will be the destination of a handoff. i.e. another agent will handoff to this agent.    

In [46]:
tools = [tool1, tool2, tool3]
handoffs = [email_writer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email to a customer', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x72947c5c2ac0>, strict_json_schema=True), FunctionTool(name='sales_agent2', description='Write a cold sales email to a customer', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x72947c5c2e80>, strict_json_schema=True), FunctionTool(name='sales_agent3', description='Write a cold sales email to a customer', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}

In [47]:
sales_manager_instructions = "You are a sales manager working for CyberAI. You use the tools given to you to generate cold sales emails. \
Use sales_agent1, sales_agent2 and sales_agent3 tools to generate 3 cold sales emails.  \
The input for each of the 3 sales_agent tools should be: \
'Write a cold sales email. Dont include any subject in your response'. \n\n\
After that select the single best email using your own judgement of which email will be most effective. \n\n\
Finally after picking the best email, you handoff (just once) to the 'Email Writer' agent to format and send that best email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

In [ ]:
# Next we will look at structrued output, and guardrails.
# structured output is a way to specify the format of the output from an agent instead of just being a string.